<a href="https://colab.research.google.com/github/spozi/gpu-svgpm/blob/main/GPFeatureGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U deap imbalanced-learn scikit-learn-intelex

     |████████████████████████████████| 160 kB 5.3 MB/s 
     |████████████████████████████████| 206 kB 9.9 MB/s 
     |████████████████████████████████| 68 kB 5.1 MB/s 
     |████████████████████████████████| 22.3 MB 33 kB/s 
     |████████████████████████████████| 15.9 MB 59 kB/s 
     |████████████████████████████████| 292.9 MB 44 kB/s 
     |████████████████████████████████| 4.1 MB 50.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [2]:
import sys
import os
import site
sys.path.append(os.path.join(os.path.dirname(site.getsitepackages()[0]), "site-packages"))

from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
#@title SVGPM Configuration
#@markdown ---
#@markdown ### Enter a file path:
train_file_path = "/content/WormsTwoClass_train.csv" #@param {type:"string"}
test_file_path = "/content/WormsTwoClass_test.csv" #@param {type:"string"}

#@markdown ---
#@markdown ### Enter SVGPM Parameters:
population_size = 500 #@param {type:"slider", min:0, max:1000, step:2}
number_of_generation = 500 #@param {type:"slider", min:0, max:1000, step:2}
#@markdown ---

In [11]:
from collections import Counter
# from imblearn.datasets import fetch_datasets
# ecoli = fetch_datasets()['ecoli']
# ecoli.data.shape

# X = ecoli.data
# y = ecoli.target

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

#Train and test file
train_file = train_file_path
test_file = test_file_path

train_data = np.loadtxt(train_file, delimiter=",", skiprows=1)
test_data = np.loadtxt(test_file, delimiter=",", skiprows=1)

X_train = train_data[:, 1:]
X_test = test_data[:, 1:]

#Normalize each feature to 0 and 1
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#y_train
y_train = train_data[:, 0].astype(int)
y_test = test_data[:, 0].astype(int)

In [12]:
print(X_train.shape, X_test.shape)

(181, 900) (77, 900)


In [ ]:
#Specify X_train, X_test, y_train, y_test here
# from sklearn.model_selection import train_test_split


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# X_train.shape

Version 2 GP Feature Generator

In [13]:
import random
import operator
import math
import statistics

import numpy as np

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

nFeatures = X_train.data.shape[1]
pset = gp.PrimitiveSet("MAIN", nFeatures) 
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.erfc, 1)
pset.addPrimitive(math.erf, 1)
pset.addPrimitive(math.exp, 1)
pset.addPrimitive(math.gamma, 1)
pset.addPrimitive(math.sqrt, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand", lambda: round(random.uniform(0.1, 1.0), 10))

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Tree", gp.PrimitiveTree)
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("main_expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=5)
toolbox.register('MAIN', tools.initIterate, creator.Tree, toolbox.main_expr)

func_cycle = [toolbox.MAIN]

toolbox.register("individual", tools.initCycle, creator.Individual, func_cycle)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [26]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import KernelPCA
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score

import warnings
warnings.filterwarnings("ignore")

def evalSymbReg(score):
  return score,

def evalSymbRegPop(population):
  # Evaluate each individual in population
  #1. Compute the expression of every individual
  list_vecs = []
  for individual in population:
    #The following code should be optimized/vectorized
    #Evaluating expression on each vector
    func = toolbox.compile(expr=individual)
    vec = []
    for x in X_train: #Iterate every vector x (row) in data (matrix) X
      try:
        val = func(*x)
        vec.append(val)
      except:
        vec.append(0)
    list_vecs.append(vec)

  #2. Convert list_vecs to numpy array
  evaluated_X = np.array(list_vecs).T
  evaluated_X = np.float32(evaluated_X)
  evaluated_X = np.nan_to_num(evaluated_X, copy=True, nan=0.0, posinf=0.0, neginf=0.0)

  #3. Individual (feature) selection
  # https://scikit-learn.org/stable/modules/feature_selection.html#feature-selection-using-selectfrommodel
  # print("Feature selection")
  # print(evaluated_X.dtype)
  clf = ExtraTreesClassifier(n_estimators=50)
  clf = clf.fit(evaluated_X, y_train)

  #4. Extract features that at top threshold (get the 75 percentile)
  # print("Feature extraction")
  q1 = np.percentile(clf.feature_importances_, 50)  #Get the top 75 percentile features
  features = [True if val >= q1 else False for val in clf.feature_importances_.tolist()] #Get the features indices
  X_train_new = evaluated_X[:, features]  #Extract the required data

  #5. Merge X_train with X_train_new row-wise
  X_train_new = np.hstack((X_train, X_train_new))

  #6. Use svc to get total nSV
  # print("SVC")
  clf_svc = SVC(C=1, kernel='linear') #Smaller C value will increase the margin size of the hyperplane. The new features should be generate such that it has very high linear seperability
  clf_svc.fit(X_train_new, y_train)
  y_pred = clf_svc.predict(X_train_new)

  #7. Compute the score
  nSV = clf_svc.support_vectors_.shape[0]
  f1 = f1_score(y_train, y_pred)
  fitness = f1/nSV

  #8. Output the fitness value
  ind_pop_fitness = []
  for f in features:
    if f is True:
      ind_pop_fitness.append(fitness) #If the feature is in the tree, set the fitness value to fitness
    else:
      ind_pop_fitness.append(0)       #If the feature is in the tree, set the fitness value to 0
  
  return ind_pop_fitness

psets = [pset]
toolbox.register("compile", gp.compileADF, psets=psets)
toolbox.register('evaluate', evalSymbReg)
toolbox.register('select', tools.selTournament, tournsize=3)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register("expr", gp.genFull, min_=1, max_=2)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=80))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=80))

In [27]:
def main():
  random.seed(1024)
  ind = toolbox.individual()
  
  pop = toolbox.population(n=population_size)
  hof = tools.HallOfFame(population_size)
  stats = tools.Statistics(lambda ind: ind.fitness.values)
  stats.register("avg", np.mean)
  stats.register("std", np.std)
  stats.register("min", np.min)
  stats.register("max", np.max)

  logbook = tools.Logbook()
  logbook.header = "gen", "evals", "std", "min", "avg", "max"

  CXPB, MUTPB, NGEN = 0.5, 0.2, number_of_generation

  # # Evaluate the entire population
#################################This is for adhoc code draft##########################################################################
  #0. Test each indvidiual in population

  # list_vecs = []
  # for ind in pop:
  #   func = toolbox.compile(expr=ind)

  #   #The following code should be optimized
  #   #Evaluating expression on each vector
  #   func = toolbox.compile(expr=ind)
  #   vec = []
  #   for x in X: #Iterate every vector x (row) in data (matrix) X
  #     val = func(*x)
  #     # try:
  #     #   val = func(*x)
  #     # except:
  #     #   print(x, [str(eq) for eq in ind])
  #       # print(ind[0])
  #     # print(val)
  #     vec.append(val)
  #   list_vecs.append(vec)

  # #2. Convert list_vecs to numpy array
  # evaluated_X = np.array(list_vecs).T
  # evaluated_X = np.nan_to_num(evaluated_X, copy=True, nan=0.0, posinf=0.0, neginf=0.0)

  # clf = ExtraTreesClassifier(n_estimators=50)
  # clf = clf.fit(evaluated_X, y)

  # #4. Extract features that at top threshold (get the 75 percentile)
  # q1 = np.percentile(clf.feature_importances_, 75)
  # print( clf.feature_importances_.tolist())
  # features = [True if val >= q1 else False for val in clf.feature_importances_.tolist()]
  # print(len(features), evaluated_X.shape)
  # new_X = evaluated_X[:, features]
  # print(new_X.shape)
  # 2/0

  # #4. 



###########################################################################################################
  #1. Compute the metric on the set of individuals
  ind_pop_fitness = evalSymbRegPop(pop)

  #2. Then, determine the best individual using toolbox
  for ind, fitness in zip(pop, ind_pop_fitness):
    ind.fitness.values = toolbox.evaluate(fitness)

  hof.update(pop)
  record = stats.compile(pop)
  logbook.record(gen=0, evals=len(pop), **record)
  print(logbook.stream)

  for g in range(1, NGEN):
    # Select the offspring
    offspring = toolbox.select(pop, len(pop))
    # Clone the offspring
    offspring = [toolbox.clone(ind) for ind in offspring]

    # Apply crossover and mutation
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        for tree1, tree2 in zip(ind1, ind2):
            if random.random() < CXPB:
                toolbox.mate(tree1, tree2)
                del ind1.fitness.values
                del ind2.fitness.values

    for ind in offspring:
        for tree, pset in zip(ind, psets):
            if random.random() < MUTPB:
                toolbox.mutate(individual=tree, pset=pset)
                del ind.fitness.values
                        
    # Evaluate the individuals with an invalid fitness
    invalids = [ind for ind in offspring if not ind.fitness.valid]

    #1. Compute the metric on the set of individuals
    ind_pop_invalid_fitness = evalSymbRegPop(invalids)

    #2. Then, determine the best individual using toolbox
    for ind, fitness in zip(invalids, ind_pop_invalid_fitness):
      ind.fitness.values = toolbox.evaluate(fitness)
            
    # Replacement of the population by the offspring
    pop = offspring
    hof.update(pop)
    record = stats.compile(pop)
    logbook.record(gen=g, evals=len(invalids), **record)
    print(logbook.stream)
  
  print('Best individual : ', hof[0][0], hof[0].fitness)
  return pop, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()

gen	evals	std        	min	avg        	max        
0  	500  	0.000158378	0  	0.000158378	0.000316756
1  	303  	0.00349813 	0  	0.00246771 	0.00775695 
2  	288  	0.00389942 	0  	0.00447069 	0.00810811 
3  	298  	0.00380091 	0  	0.00538888 	0.00817041 
4  	291  	0.00383059 	0  	0.00573932 	0.00849618 
5  	285  	0.00372525 	0  	0.00567232 	0.00849618 
6  	319  	0.00391554 	0  	0.00559719 	0.00849618 
7  	313  	0.00410051 	0  	0.00593294 	0.00913771 
8  	304  	0.00408381 	0  	0.00593957 	0.00913771 
9  	286  	0.00439775 	0  	0.00373798 	0.00913771 
10 	302  	0.00446756 	0  	0.00546502 	0.00928105 
11 	298  	0.00446603 	0  	0.00352448 	0.00928105 
12 	308  	0.00459705 	0  	0.00576942 	0.00962001 
13 	279  	0.00428427 	0  	0.006282   	0.00962001 
14 	305  	0.00425145 	0  	0.00617088 	0.00962001 
15 	284  	0.00442761 	0  	0.00682092 	0.0102124  
16 	300  	0.00418224 	0  	0.00590793 	0.0102124  
17 	292  	0.00467842 	0  	0.00380599 	0.0102124  
18 	282  	0.00468119 	0  	0.0060229  	0.0102124  


**Prediction task**

1.   Create new data from the list of fittest individual (fittest features) for both training and testing data.
2.   Fit the svm with transformed training data
3.   Predict the transformed testing data



In [28]:
#1. Evaluate the expression for training set
list_train_vecs = []
for individual in hof:
  if individual.fitness.values[0] > 0:
      func = toolbox.compile(expr=individual)
      vec = []
      for x_train in X_train: #Iterate every vector x (row) in data (matrix) X
        try:
          val = func(*x_train)
          vec.append(val)
        except:
          vec.append(0)
      list_train_vecs.append(vec)

#2. Evaluate the expression for testing set
list_test_vecs = []
for individual in hof:
  if individual.fitness.values[0] > 0:
      func = toolbox.compile(expr=individual)
      vec = []
      for x_test in X_test: #Iterate every vector x (row) in data (matrix) X
        try:
          val = func(*x_test)
          vec.append(val)
        except:
          vec.append(0)
      list_test_vecs.append(vec)

#2. Convert list_vecs to numpy array
X_train_new = np.array(list_train_vecs).T   #Need to refactor X_train g
X_train_new = np.nan_to_num(X_train_new, copy=True, nan=0.0, posinf=0.0, neginf=0.0)
X_train_new = np.hstack((X_train, X_train_new))

X_test_new = np.array(list_test_vecs).T   #Need to refactor X_test g
X_test_new = np.nan_to_num(X_test_new, copy=True, nan=0.0, posinf=0.0, neginf=0.0)
X_test_new = np.hstack((X_test, X_test_new))


#3. Fit the SVM
list_accuracy_result = []
list_f1_result = []
for c in range(-5,6):
  for g in range(-4,6):
    clf_svc = SVC(C=2**c, gamma=2**g)
    clf_svc.fit(X_train, y_train)
    y_pred = clf_svc.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    list_accuracy_result.append(accuracy)
    f1 = f1_score(y_test, y_pred)
    list_f1_result.append(f1)

print("SVM - Highest Accuracy-Score: ", max(list_accuracy_result))
print("SVM - Highest F1-Score: ", max(list_f1_result))


#4. Fit the SVM (SVGPM)
list_accuracy_result = []
list_f1_result = []
for c in range(-5,6):
  for g in range(-4,6):
    clf_svc = SVC(C=2**c, gamma=2**g)
    clf_svc.fit(X_train_new, y_train)
    y_pred = clf_svc.predict(X_test_new)
    accuracy = accuracy_score(y_test, y_pred)
    list_accuracy_result.append(accuracy)
    f1 = f1_score(y_test, y_pred)
    list_f1_result.append(f1)

print("SVGPM V2 - Highest Accuracy-Score: ", max(list_accuracy_result))
print("SVGPM V2 - Highest F1-Score: ", max(list_f1_result))


SVM - Highest Accuracy-Score:  0.6233766233766234
SVM - Highest F1-Score:  0.6133333333333334
SVGPM V2 - Highest Accuracy-Score:  0.5714285714285714
SVGPM V2 - Highest F1-Score:  0.66
